In [23]:
#pip install transformers torch colbert datasets pandas scikit-learn


In [24]:
#pip install git+https://github.com/stanford-futuredata/ColBERT.git


In [25]:
import faiss
import pandas as pd
article_df = pd.read_csv('./src/article_scraping/data/articles_with_text.csv')
print(f"Total articles in CSV: {len(article_df)}")


Total articles in CSV: 4354


In [26]:
import pandas as pd

# Load data
article_df = pd.read_csv('./src/article_scraping/data/articles_with_text.csv')

# Create a unique document ID for each article
article_df['doc_id'] = range(len(article_df))

# Replace newline characters in article text to avoid formatting issues
article_df['article_text_raw'] = article_df['article_text_raw'].str.replace(r'\n', ' ', regex=True)

# Separate rows with text from those without
with_text = article_df[~article_df['article_text_raw'].isnull() & (article_df['article_text_raw'].str.strip() != '')]
without_text = article_df[article_df['article_text_raw'].isnull() | (article_df['article_text_raw'].str.strip() == '')]

# Prepare the formatted text column for rows with text
with_text['formatted_text'] = with_text['doc_id'].astype(str) + '\t' + with_text['article_text_raw']

# Save the .tsv file (only articles with text)
with_text['formatted_text'].to_csv('./src/article_scraping/data/collection.tsv', index=False, header=False, sep='\t')

# Print a summary of the process
print(f"Articles at start: {len(article_df)}")
print(f"Articles with text: {len(with_text)}")
print(f"Articles without text: {len(without_text)}")




Articles at start: 4354
Articles with text: 4268
Articles without text: 86


/var/folders/x1/v8sbf2px7bd3mh5s_v4zrxp40000gn/T/ipykernel_3807/2489671770.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_text['formatted_text'] = with_text['doc_id'].astype(str) + '\t' + with_text['article_text_raw']


In [28]:
# from colbert.infra import Run, RunConfig, ColBERTConfig
# from colbert import Indexer
# import logging
# import os

# # Logging setup
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
# logger = logging.getLogger()

# if __name__ == '__main__':
#     with Run().context(RunConfig(nranks=4, experiment="dsan5400_project")):
        
#         # Configure ColBERT
#         config = ColBERTConfig(
#             nbits=2,
#             root="./experiments",
#             bsize=32,
#             ncells=2 
#         )
        
#         # Initialize Indexer
#         indexer = Indexer(checkpoint="checkpoints/colbertv2.0", config=config)
        
#         # Wrapper to log progress
#         def logging_index(name, collection, **kwargs):
#             if not os.path.exists(collection):
#                 logger.error(f"Collection file '{collection}' does not exist.")
#                 return
            
#             # Count total documents for progress tracking
#             with open(collection, 'r', encoding='utf-8') as file:
#                 total_docs = sum(1 for _ in file)
#             logger.info(f"Starting indexing for {total_docs} documents.")
            
#             processed_docs = 0
#             batch_size = config.bsize  
            
#             with open(collection, 'r', encoding='utf-8') as file:
#                 batch = []
#                 for line in file:
#                     batch.append(line.strip())
#                     processed_docs += 1
                    
#                     # Process batch once it reaches the batch size
#                     if len(batch) == batch_size:
#                         logger.info(f"Processing batch {processed_docs}/{total_docs}...")
#                         indexer.index(name=name, collection=batch, **kwargs)
#                         batch = []  # Clear batch after processing
                    
#                 # Process remaining lines
#                 if batch:
#                     logger.info(f"Processing final batch {processed_docs}/{total_docs}...")
#                     indexer.index(name=name, collection=batch, **kwargs)
            
#             logger.info("Indexing complete!")

#         # Run the logging wrapper
#         logging_index(
#             name="dsan5400_project.nbits=2",
#             collection="./src/article_scraping/data/collection copy.tsv",
#             overwrite=True
#         )


In [ ]:
# SIMPLIFIED VERSION FROM MEDIUM.COM

from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 500 # truncate passages at 300 tokens

index_name = 'dsan5400_project.nbits=2'
checkpoint = "checkpoints/colbertv2.0"

collection = ["Chisinau accuses Moscow of undermining democratic process and condemned drones flown over its territory. Moldova has handed a note of protest to the Russian ambassador to Chisinau over alleged interference in its recent elections. The foreign ministry in Chisinau said in a statement on Tuesday that it turned over the “note of firm protest” in relation to the “illegal and deliberate interference” to envoy Oleg Ozerov during a meeting at its offices. Moldova has accused Russia of seeking to influence its recent presidential election and referendum on joining the European Union.     Russia sought to affect results and delegitimise the democratic process, the ministry complained. Chisinau accused Russia of organising ineligible voting, bribery, and security threats in a bid to influence the votes. Moscow denies trying to sway the election, which was narrowly won by pro-Western President Maia Sandu, aided by votes from Moldovans outside the country. Moldova also narrowly voted in favour of inserting a clause in its constitution to define EU membership as a goal. Russia has maintained that the votes were unfair and does not view Sandu as the legitimate president. The pivot toward the West by Moldova, formerly part of the Soviet Union, has led to worsening relations with Russia in recent years. The European bloc, along with the United States, have backed Moldova’s assertions that Russia tried to interfere in the country’s elections. Moldova’s Ministry of Foreign Affairs also reported that it used the meeting to strongly condemn a violation of its air space by two alleged Russian drones on Sunday. “The competent institutions of our country possess clear information indicating that the drones were launched by the Russian Federation,” the ministry said. “The ambassador was warned that such aggressive incursions constitute a serious violation and an unfriendly act by the Russian Federation, as these drones endanger the lives of citizens.” Ozerov denied he had been summoned by the ministry, suggesting that the meeting was mutually agreed. “The conversation made it possible to clarify issues related to our acute and complex bilateral relations,” the Russian ambassador said. Ozerov said there was no evidence the drones were Russian and that Moscow did not fly drones through countries neighbouring Ukraine. No casualties were reported after Moldova said it found two Russian “decoy” drones crashed in two villages after a Russian drone attack on Ukraine.     ",
              "Former prime minister Navin Ramgoolam has won a third term in the Indian Ocean nation, official results show. Opposition leader Navin Ramgoolam has won a landslide victory in Mauritius’s parliamentary vote, the election commission said. The Office of the Electoral Commissioner said on Tuesday that Rangoolam and his Alliance of Change (ADC) coalition won 62.6 percent of the vote, securing the three-time former prime minister a fourth term.     The ADC won 60 of 62 seats in the National Assembly, state broadcaster Mauritius Broadcasting Corporation reported. “The court of the people has delivered its verdict and a new Mauritius awakes,” 77-year-old Rangoolam said to crowds of supporters amid boisterous cheers and blaring horns. Ramgoolam served as prime minister from 1995 to 2000 and again from 2005 to 2014. As well as the 60 seats for Mauritius, there were two up for grabs for the island of Rodrigues and another eight allocated under what is dubbed the “best loser” system. The winner-takes-all election model means single coalitions often dominate parliament. Incumbent Prime Minister Pravind Jugnauth had conceded on Monday that his Lepep alliance, led by his Militant Socialist Movement (MSM), was “heading towards a huge defeat”. “The people have chosen another team to lead the country,” Jugnauth, who has served as prime minister since 2017, told reporters. Only last month, the 62-year-old prime minister was celebrating a historic deal that saw Britain cede sovereignty over the Chagos Islands to Mauritius after a long-running dispute. But the vote was overshadowed by an explosive wire-tapping scandal, when secretly recorded phone calls of politicians, diplomats, members of civil society and journalists were leaked online. Ramgoolam said that following his landslide victory, he would first dismantle the country’s “spying system so that Mauritians will be free to talk”. During the campaign, both camps promised to improve the lot of Mauritians who face cost-of-living difficulties despite robust economic growth. Measures outlined in the Alliance of Change manifesto include the creation of a fund to support families facing hardship, free public transport, increased pensions and reduced fuel prices, as well as efforts to tackle corruption and boost the green economy. It also called for constitutional and electoral reforms including changing how the president and parliament speaker are chosen. At least one million people were registered to vote on Sunday in the 12th election since Mauritius gained independence from the United Kingdom in 1968. Mauritius, which sits about 2,000km (1,240 miles) off Africa’s east coast, is recognised as one of the continent’s most stable democracies and has developed a successful economy underpinned by its finance, tourism and agricultural sectors since gaining independence. Both Jugnauth and Ramgoolam are members of the dynasties that have dominated the leadership of Mauritius since independence. Ramgoolam, who previously worked as a doctor and a lawyer, is the son of Seewoosagur Ramgoolam, who led Mauritius to independence from Britain.",
              "The US faces a pivotal election on November 5. Two years after Roe v. Wade was overturned by a conservative-led Supreme Court, abortion access and women’s reproductive rights are shaping the presidential race. Also on the ballot are every seat in the House of Representatives and about a third of the Senate. In this episode of Now You Know, we speak with Dr. Kristin Lyerly, an OB/GYN specialising in reproductive health and a candidate for the House. She discusses the ongoing struggle for reproductive freedom in a divided country.",
              "Japanese voters go to the polls after new Prime Minister Shigeru Ishiba called a snap election. The outcome could strengthen him, or further weaken the governing LDP. Voters in Japan head to the polls on Sunday to elect members of their House of Representatives in an election seen as a test for the country’s new Prime Minister Shigeru Ishiba. With Ishiba’s governing Liberal Democratic Party (LDP) embroiled in scandals and facing diminishing public support, the vote is expected to present the party with its toughest electoral challenge in more than a decade.     While the result may be seen as a measure of public endorsement of or displeasure with Ishiba, the election is not likely to see his LDP – which has maintained a tight grip on power in Japan since 1955 – fall too far from its pedestal. Analysts expect the opposition Constitutional Democratic Party of Japan (CDPJ) to gain significant ground, but not enough to change the government. The LDP, they predict, may lose a few dozen seats. But even in a worst-case scenario, the party will still likely be number one in the ruling bloc. Here’s what you should know about Japan’s election: The LDP has ruled Japan for almost all of the post-war era and holds a majority in the 465-seat lower house. The LDP’s longtime coalition partner is Komeito, a party backed by a large Buddhist group that has often lent crucial campaign support to its political partner. Formed in 1955 and credited with leading Japan’s economic recovery after World War II, the LDP’s rule was interrupted twice, in 1993-1994 and 2009-2012. In both times, bribery scandals rocked the party and its public support. Now the LDP’s popularity has hit a low again. A recent poll by Japan’s Asahi newspaper showed the LDP may struggle in the election, potentially losing 50 of the 247 seats it now has in parliament. The main opposition CDPJ is making inroads, with the Asahi poll estimating it could grab as many as 140 seats in the election, up from its current 98. If that happens, the new prime minister’s calling of this snap election will have backfired. Other surveys portend bad news for the LDP too. According to the Pew Research Centre, just 30 percent of Japanese people surveyed in March had a favourable view of the LDP, with 68 percent holding an unfavourable view. But the opposition did not fare any better in the public’s opinion, with just 29 percent of those surveyed holding a positive view of the CDPJ, according to Pew. More concerning, only a third of those surveyed by Pew were satisfied with “the way democracy was working” in Japan. Ishiba dissolved parliament and called an election shortly after taking over as prime minister on October 1, when he replaced the LDP’s outgoing and embattled premier Fumio Kishida. Craig Mark, adjunct professor at Hosei University in Tokyo, said Ishiba called the election a year before one was required under Japan’s constitution in order to catch the opposition “off guard and secure a more solid mandate to pursue his policy agenda”. “He’s banking on the public rallying behind a new face and image for his party, following the unpopularity of former Prime Minister Fumio Kishida,” Mark wrote in The Conversation magazine. Kishida’s popularity had plummeted amid a major corruption scandal involving unreported political funds.      The opposition CDPJ, Mark said, is also hoping to increase its vote by projecting “an image of reliability and stability”. “Ishiba’s challenge in this early election is not only to win enough votes to retain government, but to be electorally successful enough to hold off his rivals from the conservative wing of the LDP,” Mark added. The Asian Network for Free Elections (ANFREL) has described the election as “crucial” for the LDP and Ishiba, in terms of gauging public trust following recent scandals and mounting economic concerns. “It will serve as a critical indicator of whether the LDP can regain public trust and retain its dominance or if opposition parties can capitalise on public dissatisfaction,” ANFREL said. Polling stations open at 7am Sunday (22:00 GMT Saturday) and voting ends at 8pm (11:00 GMT) on Sunday, with results filtering in later in the night and continuing into the early morning. Vote counting in Japan’s elections is generally conducted quickly, said Rob Fahey of The Waseda Institute for Advanced Study in Tokyo, and results will likely be announced on Sunday night, with only some seats – those that require recounts or involve other issues – being announced on Monday. If the LDP is unable to retain its poll position in the ruling coalition, questions will be asked of Ishiba’s leadership, raising the spectre of continuing political instability in Japan at a time of economic uncertainty and a challenging foreign relations environment. Analysts, in particular, point to the health of Japan’s defensive capabilities amid growing regional tension with nearby China, Russia and North Korea.      On the other hand, if the likely reduction in LDP seats “is as small as possible”, Ishiba will strengthen his standing in the party by having delivered a positive election result and will be recognised as the “prime minister who has the public’s support”, said Kazuto Suzuki, associate fellow at the Asia-Pacific Programme of Chatham House. “If Ishiba can create a secure base of government, Japanese politics will be stabilized and Japan’s foreign and security policies, which were strengthened by the Abe and Kishida administrations, can continue to be bolstered,” Suzuki wrote in an analysis brief earlier this month."]

with Run().context(RunConfig(nranks=1, experiment='notebook')):  
    # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=8) 
    # kmeans_niters specifies the number of iterations of k-means clustering.
    # 4 is a good and fast default.
    # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)

indexer.get_index()  ## This will provide the complete path of index stored on disk.